## Veri Gereksinimleri

Belirli bir tarifin veya yemeğin ait olduğu mutfağı belirleme sürecini otomatikleştirmek istiyoruz ve tarifin veya yemeğin bileşenlerini kullanarak potansiyel olarak bunun mümkün olabileceğini belirledik. Bir model oluşturabilmek için, farklı mutfaklar ve tariflerle ilgili kapsamlı verilere ihtiyacımız var.


## Veri Toplama

Bu aşamada veri bilimciler mevcut veri kaynaklarını belirler ve toplar. Biz Veri Bilimi Metodolojisinin uygulanmasına odaklanacağımız için kullanacağımız verileri hazır olarak alacağız. Bu uygulama için Yong-Yeol Ahn adlı bir araştırmacı, üç farklı web sitesinden on binlerce yemek tarifini (mutfaklar ve malzemeler) topladı.

* www.allrecipes.com
* www.epicurious.com
* www.menupan.com

Bu sayede bizim iş anlayışı aşamasında tanımlanan hedefe ulaşmak için ihtiyaç duyduğumuz veriler hazır olduğu için, herhangi bir veri toplama işlemi gerçekleştirmemize gerek kalmayacak. Şu adresten verileri çalışma klasörünüze yani bu dosyanın kayıtlı olduğu klasör içine indirebilirsiniz: https://obenseven.com.tr/recipes.csv


Şimdi Pyhon kodu içinden veriyi okuyalım;

In [39]:
import pandas as pd # verileri panda kütüphanesi ile okuyacağız
pd.set_option('display.max_columns', None)

tarifler = pd.read_csv("recipes.csv")

Verilerin başarıyla indirilip, kullanılmak üzere hazır olduğunu teyit etmek için ilk bir kaç veri satırını görüntüleyelim;

In [ ]:
tarifler.head()

Elimizde ne kadar veri olduğunu görelim;

In [ ]:
tarifler.shape

Veri setimiz 57.691 tariften oluşmaktadır. Her satır bir tarifi temsil eder ve her tarif için, ilgili mutfağın yanı sıra tarifte badem ile başlayıp kabak ile biten 384 malzemenin bulunup bulunmadığı belgelenir.

## Veriyi Anlama

Veri anlama, veri setinin oluşturulmasıyla ilgili tüm faaliyetleri kapsar. Temel olarak, veri bilimi metodolojileri içinde yer alan veri anlama bölümü, “Topladığınız veriler çözülmesi gereken sorunu temsil ediyor mu?” sorusuna cevap verir.

Verileri anlamak için modelde değişken olacak veri sütunlarında tanımlayıcı istatistiklerin hesaplanması gerekir. Bunlar ilk olarak; Hurst, Tek Değişken Analizi(Univariate Analysis) gibi istatistikler ile ortalama, medyan, minimum, maksimum ve standart sapma gibi her bir değişkene ilişkin istatistikleri içerir.

İkinci olarak, belirli değişkenlerin ne kadar yakından ilişkili olduğunu ve varsa hangilerinin çok yüksek düzeyde ilişkili olduğunu görmek için ikili korelasyonlar(Pairwise Correlations) kullanılır. Ki bu sayede, bu değişkenlerin hangilerinin esasen gereksiz olacağını tespit etmemizi sağlayarak, modelleme için yalnızca hangisinin alakalı olduğunu ve kullanılacağını tespit etmemizi sağlar.

Üçüncü olarak, dağılımlarını anlamak için değişkenlerin histogramları incelenir Histogramlar, değerlerin veya bir değişkenin dağılımını ve değişkeni modelde daha kullanışlı hale getirmek için ne tür bir veri hazırlığının gerekli olabileceğini anlamanın iyi bir yoludur. Örneğin, çok farklı değerleri olan bir kategorik değişkenin bir model içinde bilgi verebiliyor olması için histogram, bu değerlerin nasıl birleştirileceğine karar verilmesine yardımcı olur.

Veri Bilimci problem ve verilerle ne kadar çok çalışırsa o kadar çok öğrenir, dolayısıyla model içinde yapılabilecek daha fazla iyileştirme keşfeder ve sonuçta probleme daha iyi bir çözüm üretir.

Veriyi Anlama aşamasında yapılacak işlemler için gerekli kütüphaneleri ekleyerek kodlamaya başlayalım. Bu aşamada Pandas kütüphanesinin yanı sıra Numpy kütüphanesinden ve metinlerle çalıştığımız için Regular Expression(Regex) kütüphanesinden de yararlanacağız:

In [42]:
import numpy as np # numpy kütüphanesini de dahil edelim
import re # metinlerle çalışmak için regular expression kütüphanesini de ekleyelim

Basitçe bir sushi tarifinde şu malzemelerin olduğunu biliyoruz;
– Pirinç
– Soya sosu
– Wasabi
– Balık ve sebzeler

Bunların veri setimiz içinde olup olmadığını kontrol edelim;

In [ ]:
icindekiler = list(tarifler.columns.values)

print([match.group(0) for malzeme in icindekiler for match in [(re.compile(".*(rice).*")).search(malzeme)] if match])
print([match.group(0) for malzeme in icindekiler for match in [(re.compile(".*(wasabi).*")).search(malzeme)] if match])
print([match.group(0) for malzeme in icindekiler for match in [(re.compile(".*(soy).*")).search(malzeme)] if match])

Gördüğünüz gibi malzemelerimiz mevcut. Öyleyse bir tarif eğer bu 3 malzemeyi de içeriyorsa o tarife büyük ihtimalle “Japon mutfağına ait bir tarifdir.” diyebiliriz. Dolayısıyla elimizdeki veri ile sorumuza cevap verebilecek bir yöntem bulduk diyebiliriz.

## Veri Hazırlama

Veri hazırlama, veriyi cevabını aradığımız soruya uygun hale getirmenin bir yoludur. Veri hazırlama aşamasında verilerin dönüştürülmesi, verilerin çalışılması daha kolay olabilecek bir duruma getirilmesi sürecidir. Diğer bir ifadeyle kusurlu olan ürünlerin elenmesi gibi düşünülebilir. Spesifik olarak, metodolojinin veri hazırlama aşamasında, verilerle etkili bir şekilde çalışabilmek ve her şeyin doğru biçimlendirildiğinden emin olmak için eksik veya geçersiz değerleri ele alarak, yinelenen verileri kaldıracak şekilde veri hazırlanmalıdır.

İlk olarak verinin temizlenmeye ihtiyacı olup olmadığına bakalım;

In [ ]:
tarifler["country"].value_counts()

Daha yukarıda head() fonksiyonu ile sadece bir bölümünü verdiğimiz tablonun tamamını incelediğinizde şu çıkarımları yapabilirsiniz;
– Mutfak sütunu Ülke olarak etiketlenmiş, onun yerine ülke mutfağı daha açıklayıcı olurdu.
– Mutfak adları tutarlı değil çünkü hepsi büyük harfle başlamıyor.
– Vietnam ve Vietnamca gibi bazı mutfaklar ülke adının varyasyonu olarak birden fazla karşımıza çıkıyor.
– en son kodumuzda yer alan value_Counts() fonksiyonu ile gördük ki bazı mutfakların çok fazla bazılarının ise çok az tarifi vardır.

Şimdi bu hataları düzeltelim.

Öncelikle ülke(country) olarak verilmiş olan sütun adını Ülke Mutfağı olarak değiştirelim;

In [ ]:
sutun_adi = tarifler.columns.values
sutun_adi[0] = "Ülke Mutfağı"
tarifler.columns = sutun_adi

tarifler

Tüm mutfak isimlerini küçük harf yapalım;

In [46]:
tarifler["Ülke Mutfağı"] = tarifler["Ülke Mutfağı"].str.lower()

Mutfak isimlerini daha anlamlı hale getirelim. Ayrıca vietnamese ve vietnam gibi tekrar eden aynı mutfakları Vietnam Mutfağı gibi aynı isim altında birleştirelim;

In [ ]:
mutfaklar = {
    'vietnamese': 'Vietnam Mutfağı',
    'irish' : 'İrlanda Mutfağı',
    'japanese' : 'Japon Mutfağı',
    'indian': 'Hint Mutfağı',
    'spanish_portuguese': 'İspanyol Mutfağı',
    'jewish': 'İsrail Mutfağı',
    'french': 'Fransız Mutfağı',
    'central_southamerican': 'Güney Amerika Mutfağı',
    'cajun_creole': 'Cajun&Creole Mutfağı',
    'thai': 'Tayland Mutfağı',
    'asian': 'Asya Mutfağı',
    'east_asian': 'Doğu Asya Mutfağı',
    'mexico': 'Meksika Mutfağı', 
    'scandinavian': 'İskandinav Mutfağı',
    'greek': 'Yunan Mutfağı',
    'american': 'Amerikan Mutfağı',
    'african': 'Afrika Mutfağı',
    'middleeastern': 'Orta Doğu Mutfağı',
    'easterneuropean_russian': 'Doğu Avrupa-Rus Mutfağı',
    'italian': 'İtalyan Mutfağı',
    'mexican': 'Meksika Mutfağı',
    'chinese': 'Çin Mutfağı',
    'german': 'Alman Mutfağı',
    'mediterranean': 'Akdeniz Mutfağı',
    'moroccan': 'Fas Mutfağı',
    'southern_soulfood': 'Afrika Kökenli Amerika Mutfağı',
    'english_scottish': 'İngiliz Mutfağı',
    'southwestern': 'Güneybatı Amerika Mutfağı',
    'western': 'Batı Amerika Mutfağı',
    'korean': 'Kore Mutfağı',
    'caribbean': 'Karayip Mutfağı',
    'eastern-europe': 'Doğu Avrupa Mutfağı',
    'south-america': 'Güney Amerika Mutfağı',
    'north-african': 'Kuzey Afrika Mutfağı',
    'canada': 'Kanada Mutfağı',
    'turkey': 'Türk Mutfağı',
    'bangladesh': 'Bangladeş Mutfağı',
    'india': 'Hint Mutfağı',
    'france': 'Fransız Mutfağı',
    'italy': 'İtalyan Mutfağı',
    'spain': 'İspanyol Mutfağı',
    'israel': 'İsrail Mutfağı',
    'japan': 'Japon Mutfağı',
    'korea': 'Kore Mutfağı',
    'south-african': 'Güney Afrika Mutfağı',
    'uk-and-ireland': 'İngiliz Mutfağı',
    'china': 'Çin Mutfağı',
    'belgium': 'Belçika Mutfağı',
    'germany': 'Alman Mutfağı',
    'switzerland': 'İsviçre Mutfağı',
    'spain': 'İspanyol Mutfağı',
    'portugal': 'Portekiz Mutfağı',
    'iran': 'İran Mutfağı',
    'pakistan': 'Pakistan Mutfağı',
    'netherlands': 'Hollanda Mutfağı',
    'scandinavia': 'İskandinav Mutfağı',
    'philippines': 'Filipin Mutfağı',
    'indonesia': 'Endonezya Mutfağı',
    'east-african': 'Doğu Afrika Mutfağı',
    'west-african': 'Batı Afrika Mutfağı',
    'vietnam': 'Vietnam Mutfağı',
    'thailand': 'Tayland Mutfağı',
    'malaysia': 'Malezya Mutfağı',
    'lebanon': 'Lübnan Mutfağı',
    'austria': 'Avusturya Mutfağı'
}

for i, tarif in enumerate(tarifler['Ülke Mutfağı']):
    if tarif in mutfaklar:
        tarifler['Ülke Mutfağı'][i] = mutfaklar[tarif]

Hatamız var mı, eksik mutfak ismi kalmış mı kontrol edelim;

In [ ]:
print(tarifler['Ülke Mutfağı'].unique())

In [ ]:
tarifler

Tekrar eden sonuçlarımız var mı kontrol edelim;

In [ ]:
tarifler_en_list = list(tarifler.columns)
tekrar_edenler_en = [item for item in set(tarifler_en_list) if tarifler_en_list.count(item) > 1]
print(tekrar_edenler_en)

In [ ]:
# İngilizce sütun isimleri ve Türkçe karşılıkları içeren bir sözlük oluştur
malzemeler_cevirisi = {
    "Ülke Mutfağı": "Ülke Mutfağı",
    'almond': 'badem',
    'angelica': 'melek otu',
    'anise': 'anason',
    'anise_seed': 'anason tohumu',
    'apple': 'elma',
    'apple_brandy': 'elma rakısı',
    'apricot': 'kayısı',
    'armagnac': 'armanyak',
    'artemisia': 'pelin otu',
    'artichoke': 'enginar',
    'asparagus': 'kuşkonmaz',
    'avocado': 'avokado',
    'bacon': 'domuz pastırması',
    'baked_potato': 'fırında patates',
    'balm': 'melisa',
    'banana': 'muz',
    'barley': 'arpa',
    'bartlett_pear': 'bartlett armudu',
    'basil': 'fesleğen',
    'bay': 'akdeniz defnesi',
    'bean': 'fasulye',
    'beech': 'kayın',
    'beef': 'sığır eti',
    'beef_broth': 'sığır suyu',
    'beef_liver': 'sığır ciğeri',
    'beer': 'bira',
    'beet': 'pancar',
    'bell_pepper': 'dolmalık biber',
    'bergamot': 'bergamot',
    'berry': 'kırmızı orman meyvesi',
    'bitter_orange': 'acı portakal',
    'black_bean': 'kara fasulye',
    'black_currant': 'siyah frenk üzümü',
    'black_mustard_seed_oil': 'kara hardal tohumu yağı',
    'black_pepper': 'karabiber',
    'black_raspberry': 'siyah ahududu',
    'black_sesame_seed': 'siyah susam tohumu',
    'black_tea': 'siyah çay',
    'blackberry': 'böğürtlen',
    'blackberry_brandy': 'böğürtlen likörü',
    'blue_cheese': 'mavi peynir',
    'blueberry': 'yaban mersini',
    'bone_oil': 'kemik yağı',
    'bourbon_whiskey': 'bourbon viskisi',
    'brandy': 'brendi',
    'brassica': 'turpgiller',
    'bread': 'ekmek',
    'broccoli': 'brokoli',
    'brown_rice': 'esmer pirinç',
    'brussels_sprout': 'brüksel lahanası',
    'buckwheat': 'kara buğday',
    'butter': 'tereyağı',
    'buttermilk': 'yayık ayranı',
    'cabbage': 'lahana',
    'cabernet_sauvignon_wine': 'cabernet sauvignon şarabı',
    'cacao': 'kakao bitkisi',
    'camembert_cheese': 'camembert peyniri',
    'cane_molasses': 'şeker kamışı pekmezi',
    'caraway': 'çemen tohumu',
    'cardamom': 'kakule',
    'carnation': 'karanfil çiçeği',
    'carob': 'keçiboynuzu',
    'carrot': 'havuç',
    'cashew': 'kaju',
    'cassava': 'manyok',
    'catfish': 'yayın balığı',
    'cauliflower': 'karnabahar',
    'caviar': 'havyar',
    'cayenne': 'cayenne biberi',
    'celery': 'kereviz',
    'celery_oil': 'kereviz yağı',
    'cereal': 'tahıl',
    'chamomile': 'papatya',
    'champagne_wine': 'şampanya',
    'chayote': 'chayote meyvesi',
    'cheddar_cheese': 'cheddar peyniri',
    'cheese': 'peynir',
    'cherry': 'kiraz',
    'cherry_brandy': 'kiraz likörü',
    'chervil': 'fransız maydanozu',
    'chicken': 'tavuk',
    'chicken_broth': 'tavuk suyu',
    'chicken_liver': 'tavuk ciğeri',
    'chickpea': 'nohut',
    'chicory': 'beyaz hindiba',
    'chinese_cabbage': 'çin lahanası',
    'chive': 'frenk soğanı',
    'cider': 'elma şarabı',
    'cilantro': 'kişniş otu',
    'cinnamon': 'tarçın',
    'citrus': 'narenciye',
    'citrus_peel': 'narenciye kabuğu',
    'clam': 'deniz tarağı',
    'clove': 'karanfil',
    'cocoa': 'toz kakao',
    'coconut': 'hindistan cevizi',
    'coconut_oil': 'hindistan cevizi yağı',
    'cod': 'morina balığı',
    'coffee': 'kahve',
    'cognac': 'konyak',
    'concord_grape': 'concord üzümü',
    'condiment': 'baharat',
    'coriander': 'toz kişniş',
    'corn': 'mısır',
    'corn_flake': 'mısır gevreği',
    'corn_grit': 'mısır unu',
    'cottage_cheese': 'çökelek',
    'crab': 'yengeç',
    'cranberry': 'turna yemişi',
    'cream': 'krema',
    'cream_cheese': 'krem peynir',
    'cucumber': 'salatalık',
    'cumin': 'kimyon',
    'cured_pork': 'işlenmiş domuz eti',
    'currant': 'frenk üzümü',
    'date': 'hurma',
    'dill': 'dereotu',
    'durian': 'durian meyvesi',
    'eel': 'yılan balığı',
    'egg': 'yumurta',
    'egg_noodle': 'yumurtalı erişte',
    'elderberry': 'mürver',
    'emmental_cheese': 'emmental peyniri',
    'endive': 'hindiba',
    'enokidake': 'enoki mantarı',
    'fennel': 'rezene',
    'fenugreek': 'çemen',
    'feta_cheese': 'beyaz peynir',
    'fig': 'incir',
    'fish': 'balık',
    'flower': 'çiçek',
    'frankfurter': 'frankfurter sosis',
    'fruit': 'meyve',
    'galanga': 'galanga otu',
    'gardenia': 'gardenya',
    'garlic': 'sarımsak',
    'gelatin': 'jelatin',
    'geranium': 'sardunya',
    'gin': 'cin',
    'ginger': 'zencefil',
    'goat_cheese': 'keçi peyniri',
    'grape': 'üzüm',
    'grape_brandy': 'üzüm likörü',
    'grape_juice': 'üzüm suyu',
    'grapefruit': 'greyfurt',
    'green_bell_pepper': 'yeşil dolmalık biber',
    'green_tea': 'yeşil çay',
    'gruyere_cheese': 'gruyere peyniri',
    'guava': 'guava',
    'haddock': 'mezgit',
    'ham': 'domuz jambonu',
    'hazelnut': 'fındık',
    'herring': 'ringa balığı',
    'holy_basil': 'kutsal fesleğen',
    'honey': 'bal',
    'hop': 'şerbetçi otu',
    'horseradish': 'yaban turpu',
    'huckleberry': 'ayı üzümü',
    'jamaican_rum': 'jamaika romu',
    'japanese_plum': 'japon eriği',
    'jasmine': 'yasemin',
    'jasmine_tea': 'yasemin çayı',
    'juniper_berry': 'ardıç meyvesi',
    'kaffir_lime': 'kaffir lime',
    'kale': 'kara lahana',
    'katsuobushi': 'kurutulmuş balık pulu',
    'kelp': 'su yosunu',
    'kidney_bean': 'kuru fasulye',
    'kiwi': 'kivi',
    'kohlrabi': 'alabaş',
    'kumquat': 'kumkuat',
    'lamb': 'kuzu eti',
    'lard': 'domuz yağı',
    'laurel': 'defne',
    'lavender': 'lavanta',
    'leaf': 'yaprak',
    'leek': 'pırasa',
    'lemon': 'limon',
    'lemon_juice': 'limon suyu',
    'lemon_peel': 'limon kabuğu',
    'lemongrass': 'limon otu',
    'lentil': 'mercimek',
    'lettuce': 'marul',
    'licorice': 'meyan kökü',
    'lilac_flower_oil': 'leylak çiçeği yağı',
    'lima_bean': 'lima fasulyesi',
    'lime': 'misket limonu',
    'lime_juice': 'misket limonu suyu',
    'lime_peel_oil': 'misket limonu kabuk yağı',
    'lingonberry': 'dağ mersini',
    'litchi': 'liçi',
    'liver': 'ciğer',
    'lobster': 'ıstakoz',
    'long_pepper': 'uzun biber',
    'lovage': 'levzine',
    'macadamia_nut': 'makademya fındığı',
    'macaroni': 'makarna',
    'mace': 'top muskat',
    'mackerel': 'uskumru',
    'malt': 'malt',
    'mandarin': 'bodrum mandalina',
    'mandarin_peel': 'mandalina kabuğu',
    'mango': 'mango',
    'maple_syrup': 'akçaağaç şurubu',
    'marjoram': 'mercanköşk',
    'mate': 'mate çayı',
    'matsutake': 'matsutake mantarı',
    'meat': 'et',
    'melon': 'kavun',
    'milk': 'süt',
    'milk_fat': 'süt yağı',
    'mint': 'toz nane',
    'mozzarella_cheese': 'mozzarella peyniri',
    'mung_bean': 'maş fasulyesi',
    'munster_cheese': 'munster peyniri',
    'muscat_grape': 'muskat üzümü',
    'mushroom': 'mantar',
    'mussel': 'midye',
    'mustard': 'hardal',
    'mutton': 'koyun eti',
    'nectarine': 'nektarin',
    'nira': 'nira bitkisi',
    'nut': 'kabuklu yemiş',
    'nutmeg': 'muskat',
    'oat': 'yulaf',
    'oatmeal': 'yulaf ezmesi',
    'octopus': 'ahtapot',
    'okra': 'bamya',
    'olive': 'zeytin',
    'olive_oil': 'zeytinyağı',
    'onion': 'soğan',
    'orange': 'portakal',
    'orange_flower': 'portakal çiçeği',
    'orange_juice': 'portakal suyu',
    'orange_peel': 'portakal kabuğu',
    'oregano': 'keklik otu',
    'ouzo': 'uzo',
    'oyster': 'istiridye',
    'palm': 'palmiye',
    'papaya': 'papaya',
    'parmesan_cheese': 'parmesan peyniri',
    'parsley': 'maydanoz',
    'parsnip': 'yaban havucu',
    'passion_fruit': 'çarkıfelek meyvesi',
    'pea': 'bezelye',
    'peach': 'şeftali',
    'peanut': 'yer fıstığı',
    'peanut_butter': 'yer fıstığı ezmesi',
    'peanut_oil': 'yer fıstığı yağı',
    'pear': 'armut',
    'pear_brandy': 'armut likörü',
    'pecan': 'pekan cevizi',
    'pelargonium': 'itır çiçeği',
    'pepper': 'biber',
    'peppermint': 'ingiliz nanesi',
    'peppermint_oil': 'nane yağı',
    'pimenta': 'yenibahar',
    'pimento': 'kiraz biberi',
    'pineapple': 'ananas',
    'pistachio': 'antep fıstığı',
    'plum': 'erik',
    'popcorn': 'patlamış mısır',
    'porcini': 'porçini mantarı',
    'pork': 'domuz eti',
    'pork_liver': 'domuz ciğeri',
    'pork_sausage': 'domuz sucuğu',
    'port_wine': 'porto şarabı',
    'potato': 'patates',
    'potato_chip': 'patates cipsi',
    'prawn': 'büyük karides',
    'prickly_pear': 'dikenli armut',
    'provolone_cheese': 'provolone peyniri',
    'pumpkin': 'balkabağı',
    'quince': 'ayva',
    'radish': 'turp',
    'raisin': 'kuru üzüm',
    'rapeseed': 'kolza tohumu',
    'raspberry': 'ahududu',
    'raw_beef': 'çiğ sığır eti',
    'red_algae': 'kırmızı yosun',
    'red_bean': 'kırmızı fasulye',
    'red_kidney_bean': 'kırmızı böbrek fasulyesi',
    'red_wine': 'kırmızı şarap',
    'rhubarb': 'ışgın',
    'rice': 'pirinç',
    'roasted_almond': 'kavrulmuş badem',
    'roasted_beef': 'kavrulmuş sığır eti',
    'roasted_hazelnut': 'kavrulmuş fındık',
    'roasted_meat': 'kavrulmuş et',
    'roasted_nut': 'kavrulmuş yemiş',
    'roasted_peanut': 'kavrulmuş yer fıstığı',
    'roasted_pecan': 'kavrulmuş pekan cevizi',
    'roasted_pork': 'kavrulmuş domuz eti',
    'roasted_sesame_seed': 'kavrulmuş susam tohumu',
    'romano_cheese': 'romano peyniri',
    'root': 'kök',
    'roquefort_cheese': 'roquefort peyniri',
    'rose': 'gül',
    'rosemary': 'biberiye',
    'rum': 'rom',
    'rutabaga': 'isveç şalgamı',
    'rye_bread': 'çavdar ekmeği',
    'rye_flour': 'çavdar unu',
    'saffron': 'safran',
    'sage': 'adaçayı',
    'sake': 'sake',
    'salmon': 'somon',
    'salmon_roe': 'somon havyarı',
    'sassafras': 'sassafras',
    'sauerkraut': 'lahana turşusu',
    'savory': 'sater',
    'scallion': 'yeşil soğan',
    'scallop': 'deniz tarağı',
    'sea_algae': 'deniz otu',
    'seaweed': 'deniz yosunu',
    'seed': 'tohum',
    'sesame_oil': 'susam yağı',
    'sesame_seed': 'susam tohumu',
    'shallot': 'arpacık soğanı',
    'sheep_cheese': 'koyun peyniri',
    'shellfish': 'kabuklu deniz ürünleri',
    'sherry': 'şeri',
    'shiitake': 'shiitake mantarı',
    'shrimp': 'karides',
    'smoke': 'duman',
    'smoked_fish': 'füme balık',
    'smoked_salmon': 'füme somon',
    'smoked_sausage': 'füme sosis',
    'sour_cherry': 'vişne',
    'sour_milk': 'ekşi süt',
    'soy_sauce': 'soya sosu',
    'soybean': 'soya fasulyesi',
    'soybean_oil': 'soya yağı',
    'spearmint': 'kıvırcık nane',
    'squash': 'sarı kabak',
    'squid': 'kalamar',
    'star_anise': 'yıldız anasonu',
    'starch': 'nişasta',
    'strawberry': 'çilek',
    'strawberry_jam': 'çilek reçeli',
    'strawberry_juice': 'çilek suyu',
    'sturgeon_caviar': 'mersin havyarı',
    'sumac': 'sumak',
    'sunflower_oil': 'ayçiçek yağı',
    'sweet_potato': 'tatlı patates',
    'swiss_cheese': 'isviçre peyniri',
    'tabasco_pepper': 'tabasco biberi',
    'tamarind': 'demirhindi',
    'tangerine': 'mandalina',
    'tarragon': 'tarhun',
    'tea': 'çay',
    'tequila': 'tekila',
    'thai_pepper': 'thai biberi',
    'thyme': 'kekik',
    'tomato': 'domates',
    'tomato_juice': 'domates suyu',
    'truffle': 'trüf',
    'tuna': 'ton balığı',
    'turkey': 'hindi',
    'turmeric': 'zerdeçal',
    'turnip': 'şalgam',
    'vanilla': 'vanilya',
    'veal': 'dana eti',
    'vegetable': 'sebze',
    'vegetable_oil': 'bitkisel yağ',
    'vinegar': 'sirke',
    'violet': 'menekşe',
    'walnut': 'ceviz',
    'wasabi': 'wasabi',
    'watercress': 'tere',
    'watermelon': 'karpuz',
    'wheat': 'buğday',
    'wheat_bread': 'buğday ekmeği',
    'whiskey': 'viski',
    'white_bread': 'beyaz ekmek',
    'white_wine': 'beyaz şarap',
    'whole_grain_wheat_flour': 'tam buğday unu',
    'wine': 'şarap',
    'wood': 'odun',
    'yam': 'yam',
    'yeast': 'maya',
    'yogurt': 'yoğurt',
    'zucchini': 'yeşil kabak'
}

# Listenin Türkçe karşılıklarla değiştirilmiş hali
turkce_malzemeler = [malzemeler_cevirisi.get(item, item) for item in tarifler.columns]
print(len(turkce_malzemeler))
print(len(list(tarifler.columns)))
tarifler.columns = turkce_malzemeler
print(list(tarifler.columns))
tarifler.head()

In [ ]:
tekrar_edenler = [item for item in set(turkce_malzemeler) if turkce_malzemeler.count(item) > 1]
print(tekrar_edenler)

50 tariften az tarife sahip olan mutfakları çıkartalım;

In [ ]:
# Verilerini tutacağımız mutfakları listeleyelim
tarif_sayisi = tarifler["Ülke Mutfağı"].value_counts()
tarif_indisleri = tarif_sayisi > 50

secilen_mutfaklar = list(np.array(tarif_sayisi.index.values)[np.array(tarif_indisleri)])

onceki_veri = tarifler.shape[0] # orjinal verideki satır sayısı
print("Orjinal verideki satır sayısı: {}.".format(onceki_veri))

tarifler = tarifler.loc[tarifler['Ülke Mutfağı'].isin(secilen_mutfaklar)]

sonraki_veri = tarifler.shape[0] # seçilen mutfaklara ait verideki satır sayısı
print("Tarif sayısı 50'den çok olan mutfakların sayısı: {}.".format(sonraki_veri))

print("{} kayıt silindi!".format(onceki_veri - sonraki_veri))

Verilerde yer alan Yes(Malzeme bu mutfakta var anlamında) ifadesini “1” ile ve No(Malzeme bu mutfakta yok anlamında) ifadesini “0” ile değiştirelim;

In [ ]:
tarifler = tarifler.replace(to_replace="Yes", value=1)
tarifler = tarifler.replace(to_replace="No", value=0)

Şimdi verileri daha iyi anlamamıza ve ne tür bilgiler edinebileceğimizi anlamamıza yardımcı olacak bazı ek analizlere göz atalım.

Örneğin, tarif içinde pirinç(rice), soya sosu(soy), wasabi ve yosun(seaweed) bulunan tarifler hangi mutfaklara ait bulalım;

In [ ]:
bunlar_hangi_mutfakta = tarifler.loc[
    (tarifler["pirinç"] == 1) &
    (tarifler["soya sosu"] == 1) &
    (tarifler["wasabi"] == 1) &
    (tarifler["deniz yosunu"] == 1)
]

bunlar_hangi_mutfakta

Yukarıdaki kkodun çıktısına bakarak tarifinde pirinç, soya sosu, wasabi ve yosun bulunan yemeklerin Japon mutfağına ait olduğunu söyleyebilir miyiz?

Tabii ki hayır, çünkü sonuçlara göre asya ve doğu asya mutfaklarında da bu malzemeler kullanılıyor.

Şimdi farklı bir sorunun cevabını arayalım;

Tüm tarifler içinde hangi malzeme kaç kez kullanılmıştır?

In [ ]:
# Tüm sütunların toplamı
mlz = tarifler.iloc[:, 1:].sum(axis=0)
print(mlz)

Biraz daha biçimlendirilmiş olarak görüntüleyelim;

In [ ]:
# sütunları panda serisi olarak tanımlayalım
malzeme = pd.Series(mlz.index.values, index = np.arange(len(mlz)))
sayim = pd.Series(list(mlz), index = np.arange(len(mlz)))

# veri setini oluşturalım
mlz_df = pd.DataFrame(dict(malzeme = malzeme, sayim = sayim))
mlz_df = mlz_df[["malzeme", "sayim"]]
print(mlz_df.to_string())

Artık dilersek bu veri setini istediğimiz şekilde sıralayabiliriz. Örneğin tüm tarifler içinde en çok kullanılan malzemelerin neler olduğunu merak ediyorsak;

In [ ]:
mlz_df.sort_values(["sayim"], ascending=False, inplace=True)
mlz_df.reset_index(inplace=True, drop=True)

print(mlz_df)

Tabii burada elde ettiğimiz sonuçlar bir miktar yanlılığa sahiptir. Çünkü yukarıdaki tablolardan hatırlayacağınız gibi tarifler arasında 40.000’in üzerinde Amerikan mutfağına ait tarif mevcut. Bu da cevap aradığımız soruların Amerikan mutfağından aşırı miktarda etkilenen cevaplar olduğunu gösterir. Bu yüzden daha objektif bir sonuç için ülke mutfağına göre bu sonucu üretebiliriz. Diğer bir deyişle şu soruların cevabını arayabiliriz; “Çinliler hangi malzemeleri daha çok kullanıyor?” ya da “Kanadalılar için en önemli malzemeler nelerdir?“ gibi;

In [ ]:
mutfaklar = tarifler.groupby("Ülke Mutfağı").mean()
mutfaklar.head()

Hadi şimdi ülke mutfaklarına göre en çok kullanılan 4 malzemeyi görelim;

In [ ]:
mlz_sayisi = 4 # çıktıya yazdırılacak malzeme sayısını belirleyelim

# her mutfağa göre en çok kullanılan malzemeleri yazdıracak fonksiyon
def en_cok_kullanilan_malzemeler(row):
    print(row.name.upper())
    sirali_satirlar = row.sort_values(ascending=False)*100
    cok_kullanilanlar = list(sirali_satirlar.index.values)[0:mlz_sayisi]
    siralanmis = list(sirali_satirlar)[0:mlz_sayisi]

    for indis, malzemeler in enumerate(cok_kullanilanlar):
        print("%s (%d%%)" % (malzemeler, siralanmis[indis]), end=' ')
    print("\n")

# fonksiyonu veri setimiz üzerinde çalıştıralım
create_cuisines_profiles = mutfaklar.apply(en_cok_kullanilan_malzemeler, axis=1)

## Modelleme

Veri Modelleme, hem tanımlayıcı hem de tahmine dayalı modeller geliştirmeye odaklanır. 

Tanımlayıcı bir model, şu tür örnekleri inceleyebilir: Eğer bir kişi bunu yaptıysa, muhtemelen bunu tercih edecektir. Tahmine dayalı bir model ise, evet/hayır gibi ikili(binary) sonuçlar vermeye çalışır. Bu modeller, istatistiksel olarak yönlendirilen veya makine öğrenimi tarafından yönlendirilen analitik yaklaşıma dayanmaktadır.


In [ ]:
# Karar ağacı modellemesi için scikit-learn kütüphanesini ekleyelim 
%matplotlib inline
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib.pyplot as plt

!conda install python-graphviz --yes
import graphviz

from sklearn.tree import export_graphviz

import itertools

Veriyi tekrar gözden geçirelim;

In [ ]:
tarifler.head

Burada, Asya (Kore, Japon, Çin, Tayland) ve Hint mutfaklarından sadece bazılarına ait tarifler için bir karar ağacı oluşturuyoruz. Bunu yapmamızın nedeni, verilerin bir mutfağa veya daha önce bahsettiğimiz gibi Amerikan mutfağından çok sayıda tarif içermesi nedeniyle bir grup mutfağa karşı ağırlıklı olması durumunda karar ağacının iyi çalışmamasıdır. Amerikan mutfaklarını analizimizin dışında tutabilir veya verilerin farklı alt kümeleri için karar ağaçları oluşturabiliriz. Biz çalışmamız için ikinci çözümü tercih edelim. Asya ve Hint mutfaklarına ilişkin verileri kullanarak karar ağacımızı oluşturalım ve karar ağacımıza dogu_agaci adını verelim.

In [ ]:
# Ülke mutfaklarına ait alt mutfakları seçelim
asya_hint_mutfaklari = tarifler[tarifler['Ülke Mutfağı'].isin(["Kore Mutfağı", "Japon Mutfağı", "Çin Mutfağı", "Tayland Mutfağı", "Hint Mutfağı"])]
mutfaklar = asya_hint_mutfaklari["Ülke Mutfağı"]
malzemeler = asya_hint_mutfaklari.iloc[:,1:]

dogu_agaci = tree.DecisionTreeClassifier(max_depth=3)
dogu_agaci.fit(malzemeler, mutfaklar)

print("Karar ağacı modeli dogu_agaci adıyla kaydedildi!")

Şimdi de karar ağacının grafiğini çizelim ve nasıl göründüğüne göz atalım;

In [ ]:
export_graphviz(dogu_agaci,
                feature_names=list(malzemeler.columns.values),
                out_file="dogu_agaci.dot",
                class_names=np.unique(mutfaklar),
                filled=True,
                node_ids=True,
                special_characters=True,
                impurity=False,
                label="all",
                leaves_parallel=False)

with open("dogu_agaci.dot") as dogu_agaci_resmi:
    dogu_agaci_grafigi = dogu_agaci_resmi.read()
graphviz.Source(dogu_agaci_grafigi)

Bu karar ağacını şu şekilde yorumlayabiliriz;

Eğer tarif kimyon ve kavrulmuş susam tohumu içeriyorsa, nişasta içermiyorsa, bir Çin Mutfağı tarifidir.

Eğer tarif kimyon içermiyor, balık ve soya sosu içeriyorsa bir Hint Mutfağı tarifidir.

Burada yorumlamayı nasıl yapacağınza dair örnekler yazdım ama kodu her çalıştrdığınızda farklı sonuçlar üretebilir.


## Model Değerlendirme

Yukarıda kullandığımız Asya-Hint Mutfağı modelimizi değerlendirmek için, veri setini eğitim seti ve test seti olarak bölmeliyiz. Sonra eğitim setini kullanarak karar ağacını kuracağız. Ardından, modeli test seti üzerinde test edecek ve modelin tahmin ettiği mutfakları gerçek mutfaklarla karşılaştıracağız.

Bu çalışmada daha önce tanımladığımız Asya ve Hint mutfakları ile ilgili verilerden oluşan veri çerçevesini(dataframe) kullanacağız. Bu seçtiğimiz mutfaklarda kaç tarif olduğuna göz atalım;


In [ ]:
asya_hint_mutfaklari["Ülke Mutfağı"].value_counts()

Her mutfağa ait tarifler içinden rastgele 30 tarifi test seti olarak çıkartalım ve dogu_test olarak adlandıralım.

Bu aşamada rasgele sayı üreterek kullanmamız gerekecek. Bunun için gerekli kütüphaneyi de ekleyelim:

In [ ]:
import random
# Veri boyutunu belirleyelim
sample_n = 30

# Her mutfaktan 30 tarif alalım
random.seed(1234) # rastgele sayı üretelim
dogu_test = asya_hint_mutfaklari.groupby("Ülke Mutfağı", group_keys=False).apply(lambda x: x.sample(sample_n))

dogu_test_malzemeler = dogu_test.iloc[:,1:] # malzemeler
dogu_test_mutfaklar = dogu_test["Ülke Mutfağı"] # karşılık gelen mutfaklar

Şimdi kodumuzu bir kontrol edelim ve her mutfakta 30 tarif olduğunu görelim;

In [ ]:
# her mutfakta 30 tarif olduğunu görelim
dogu_test["Ülke Mutfağı"].value_counts()

Şimdi de eğitim setimizi oluşturalım. bunun için dogu veri setimiz içinden test verisi olarak ayırdığımız 30 tarifi çıkartıyoruz ve dogu_egitim olarak isimlendiriyoruz.

In [80]:
dogu_test_index = asya_hint_mutfaklari.index.isin(dogu_test.index)
dogu_egitim = asya_hint_mutfaklari[~dogu_test_index]

dogu_egitim_malzemeler = dogu_egitim.iloc[:,1:] # malzemeler
dogu_egitim_mutfaklar = dogu_egitim["Ülke Mutfağı"] # karşılık gelen mutfaklar

Şimdi yapacağımız kontrolde eğitim veri setimizin tarif sayılarının önceki dogu veri setimizden 30 eksik olduğunu görmeliyiz;

In [ ]:
dogu_egitim["Ülke Mutfağı"].value_counts()

Bu adımda, eğitim setimizi kullanarak karar ağacı oluşturacağız. Bu yeni karar ağacını da dogu_egitim_agaci olarak isimlendirelim;

In [ ]:
dogu_egitim_agaci = tree.DecisionTreeClassifier(max_depth=15)
dogu_egitim_agaci.fit(dogu_egitim_malzemeler, dogu_egitim_mutfaklar)

print("dogu_egitim_agaci isimli karar ağacı modeliniz oluşturulmuştur!")

Şimdi bu karar ağacına göz atalım;

In [ ]:
export_graphviz(dogu_egitim_agaci,
                feature_names=list(dogu_egitim_malzemeler.columns.values),
                out_file="dogu_egitim_agaci.dot",
                class_names=np.unique(dogu_egitim_mutfaklar),
                filled=True,
                node_ids=True,
                special_characters=True,
                impurity=False,
                label="all",
                leaves_parallel=False)

with open("dogu_egitim_agaci.dot") as dogu_egitim_agaci_resmi:
    dogu_egitim_agaci_grafigi = dogu_egitim_agaci_resmi.read()
graphviz.Source(dogu_egitim_agaci_grafigi)

Şimdi modelimizi test verisi ile test edelim;

In [85]:
dogu_mutfaklar_tahmini = dogu_egitim_agaci.predict(dogu_test_malzemeler)

Karar ağacının her bir tarifin mutfağını ne kadar doğru belirleyebildiğini ölçmek için, her bir mutfaktan kaç tarifin doğru sınıflandırıldığına dair güzel bir özet sunan bir Karışıklık Matrisi(Confusion Matrix) oluşturacağız. Bu matris ayrıca hangi mutfakların başka hangi mutfaklarla karıştırıldığına da ışık tutuyor.

In [ ]:
test_mutfaklari = np.unique(dogu_test_mutfaklar)
dogu_karisiklik_matrisi = confusion_matrix(dogu_test_mutfaklar, dogu_mutfaklar_tahmini, labels = test_mutfaklari)
title = 'Doğu Mutfakları Karışıklık Matrisi(Confusion Matrix)'
cmap = plt.cm.Blues

plt.figure(figsize=(8, 6))
dogu_karisiklik_matrisi = (
    dogu_karisiklik_matrisi.astype('float') / dogu_karisiklik_matrisi.sum(axis=1)[:, np.newaxis]
    ) * 100

plt.imshow(dogu_karisiklik_matrisi, interpolation='nearest', cmap=cmap)
plt.title(title)
plt.colorbar()
tick_marks = np.arange(len(test_mutfaklari))
plt.xticks(tick_marks, test_mutfaklari)
plt.yticks(tick_marks, test_mutfaklari)

fmt = '.2f'
thresh = dogu_karisiklik_matrisi.max() / 2.
for i, j in itertools.product(range(dogu_karisiklik_matrisi.shape[0]), range(dogu_karisiklik_matrisi.shape[1])):
    plt.text(j, i, format(dogu_karisiklik_matrisi[i, j], fmt),
             horizontalalignment="center",
             color="white" if dogu_karisiklik_matrisi[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('Doğru Etiketler')
plt.xlabel('Tahmini Etiketler')

plt.show()

Satırlar veri kümesindeki gerçek mutfakları, sütunlar ise tahmin edilen mutfakları temsil eder. Her satırın toplamı %100 olmalıdır. Bu karışıklık matrisine göre aşağıdaki gözlemleri yapabiliriz:

- Karışıklık matrisindeki ilk satıra göz attığımızda görüyoruz ki, dogu_test verisi içindeki Çince tariflerin %60’ı karar ağacımız tarafından doğru sınıflandırılırken, Çince tariflerin %37’si Kore ve %3’ü Hint Mutfağı olarak yanlış sınıflandırılmıştır.
- Hint satırına göz attığımızda ise, dogu_test verisi içindeki Hint tariflerinin %77’si karar ağacımız tarafından doğru sınıflandırılmış, ancak Hint tariflerinin %3’ü Çin, %13’ü Kore ve %7’si Tayland olarak yanlış sınıflandırılmıştır.